In [1]:
# variables = ['datadate', 'ticker', 'agr', 'bm', 'mom12m', 'mve', 'operprof', 'roeq', 'absacc', 'acc', 'aeavol', 'age', 'baspread', 'BETA', 'bm_ia', 
#              'cash', 'cashdebt', 'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chfeps', 'chinv', 'chmom', 'chnanalyst', 
#              'chpmia', 'chtx', 'cinvest', 'convind', 'currat', 'depr', 'disp', 'divi', 'divo', 'dy', 'ear', 'egr', 'ep', 'fgr5yr', 
#              'gma', 'grcapx', 'grltnoa', 'herf', 'hire', 'idiovol', 'ill', 'indmom', 'invest', 'IPO', 'lev', 'mom1m', 'mom36m', 'ms', 
#              'mve_ia', 'nanalyst', 'nincr', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr', 'pchgm_pchsale', 'pchsale_pchinvt', 
#              'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc', 'pricedelay', 'ps', 'rd', 'rd_mve', 'rd_sale', 'realestate', 
#              'retvol', 'roaq', 'roavol', 'roic', 'rsup', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind', 'sfe', 'sgr', 'sin', 
#              'sp', 'std_dolvol', 'std_turn', 'stdcf', 'sue', 'tang', 'tb', 'turn', 'zerotrade']
variables = ['datadate', 'ticker', 'bm', 'mom12m', 'mve']

In [2]:
# import pandas as pd
# import numpy as np

# green_cleaned = pd.read_csv('../green cleaned.csv')
# green_cleaned['ret_fwd_1'] = (green_cleaned.groupby('permno')['ret_excess'].shift(-1) )

In [3]:
# green_cleaned = green_cleaned[green_cleaned['datadate'] < '2020-01-31']
# green_cleaned.loc[green_cleaned['datadate'] == '2019-12-31', 'ret_fwd_1'] = np.nan

In [4]:
# green_cleaned.to_csv('green_cleaned_pre_2020.csv', index=False)

In [5]:
from dotenv import find_dotenv, load_dotenv
from getpass import getpass
import os

#load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Enter your GEMINI_API_KEY: ········


In [7]:
import sqlite3
import pandas as pd
from crewai.tools import tool
from crewai import Agent, Task, Crew, Process
from agentics import AG
import time
from functools import wraps
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

# ============================================
# Rate Limiting Setup
# ============================================
class RateLimiter:
    def __init__(self, calls_per_minute=5):
        self.calls_per_minute = calls_per_minute
        self.interval = 60.0 / calls_per_minute
        self.last_call = 0
        self.call_count = 0
        
    def wait_if_needed(self):
        """Wait if necessary to respect rate limit."""
        current_time = time.time()
        time_since_last = current_time - self.last_call
        
        if time_since_last < self.interval:
            sleep_time = self.interval - time_since_last
            print(f"[Rate Limit] Waiting {sleep_time:.2f}s (Call #{self.call_count + 1})")
            time.sleep(sleep_time)
        
        self.last_call = time.time()
        self.call_count += 1

rate_limiter = RateLimiter(calls_per_minute=5)

def rate_limit(func):
    """Decorator with retry logic for rate limiting."""
    @wraps(func)
    @retry(
        retry=retry_if_exception_type((Exception,)),
        wait=wait_exponential(multiplier=1, min=4, max=60),
        stop=stop_after_attempt(5)
    )
    def wrapper(*args, **kwargs):
        rate_limiter.wait_if_needed()
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"[Error] {func.__name__} failed: {str(e)}")
            raise
    return wrapper

# ============================================
# Database Setup
# ============================================
def setup_firm_database(csv_path, db_path='firm_data.db', selected_columns=None):
    """Load your CSV into SQLite database (run this once)."""
    print(f"Loading CSV from {csv_path}...")
    
    sample_df = pd.read_csv(csv_path, nrows=5)
    
    if selected_columns is None:
        print(f"\nAvailable columns ({len(sample_df.columns)} total):")
        for i, col in enumerate(sample_df.columns, 1):
            print(f"  {i}. {col}")
        
        response = input("\nPress Enter to use ALL columns, or type 'stop' to exit: ").strip()
        if response.lower() == 'stop':
            print("Exiting.")
            return
        selected_columns = sample_df.columns.tolist()
    else:
        missing = [col for col in selected_columns if col not in sample_df.columns]
        if missing:
            print(f"\n❌ Error: These columns don't exist: {missing}")
            return
    
    clean_columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_')
                    .replace('(', '').replace(')', '').replace('/', '_') 
                    for col in selected_columns]
    
    import os
    if os.path.exists(db_path):
        os.remove(db_path)
    
    chunk_size = 50000
    conn = sqlite3.connect(db_path)
    
    first_chunk = True
    chunk_num = 0
    total_rows = 0
    
    print("\nLoading data...")
    for chunk in pd.read_csv(csv_path, chunksize=chunk_size, usecols=selected_columns):
        chunk.columns = clean_columns
        chunk_num += 1
        total_rows += len(chunk)
        
        if first_chunk:
            chunk.to_sql('firms', conn, if_exists='replace', index=False)
            first_chunk = False
        else:
            chunk.to_sql('firms', conn, if_exists='append', index=False)
        
        print(f"  Chunk {chunk_num}: {total_rows:,} rows")
    
    cursor = conn.cursor()
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_ticker ON firms(ticker)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_date ON firms(datadate)")
    conn.commit()
    conn.close()
    
    print(f"\n✓ Database created: {db_path} ({total_rows:,} rows)")
    return clean_columns

# ============================================
# Database Query Tools
# ============================================
@tool("query_firm_database")
@rate_limit
def query_firm_database(sql_query: str) -> str:
    """
    Execute SQL queries on the S&P 500 firm characteristics database.
    
    Available table: 'firms' (contains all firm characteristic data)
    
    IMPORTANT DATA NOTES:
    - 'mve' column = log(market value of equity), i.e., log size
    - ALL features are standardized: mean = 0, standard deviation = 1
    - Higher values = more extreme positive, lower values = more extreme negative
    
    Example queries:
    - SELECT * FROM firms WHERE datadate = '2023-12-31'
    - SELECT AVG(bm), AVG(mom12m), AVG(mve) FROM firms WHERE datadate = '2023-12-31'
    - SELECT ticker, bm, mom12m FROM firms WHERE datadate = '2023-12-31' AND bm < -1 ORDER BY bm LIMIT 20
    
    Args:
        sql_query: A valid SQL SELECT query
    
    Returns:
        Query results as a formatted string
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        df = pd.read_sql_query(sql_query, conn)
        conn.close()
        
        if df.empty:
            return "Query returned no results."
        
        result = f"Query returned {len(df)} rows:\n\n"
        result += df.to_string(index=False)
        
        return result
    except Exception as e:
        return f"Error executing query: {str(e)}"

@tool("get_database_schema")
@rate_limit
def get_database_schema() -> str:
    """
    Get the schema of the firms database including all column names and types.
    
    CRITICAL DATA INFORMATION:
    - 'mve' = log(market value of equity), represents log firm size
    - ALL features are standardized to mean=0, std=1
    - Values represent standard deviations from mean (z-scores)
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        cursor = conn.cursor()
        
        cursor.execute("PRAGMA table_info(firms)")
        columns = cursor.fetchall()
        
        cursor.execute("SELECT COUNT(*) FROM firms")
        row_count = cursor.fetchone()[0]
        
        cursor.execute("SELECT MIN(datadate), MAX(datadate) FROM firms")
        date_range = cursor.fetchone()
        
        conn.close()
        
        schema = f"""Database: firm_data.db
Table: firms
Total rows: {row_count:,}
Date range: {date_range[0]} to {date_range[1]}

DATA STANDARDIZATION:
- 'mve' column = log(market value of equity) - represents log firm size
- 'bm' column = log(book to market) - a high bm value means a firm is undervalued, and a low bm value means a firm is overvalued
- 'mom12m' column = 12 month momentum
- ALL features are standardized: mean = 0, standard deviation = 1
- Interpret values as z-scores (standard deviations from mean)
- Example: A value of 2.0 means 2 standard deviations above average
- Example: A value of -1.5 means 1.5 standard deviations below average

Columns ({len(columns)} total):
"""
        for col in columns:
            schema += f"  - {col[1]} ({col[2]})\n"
        
        return schema
    except Exception as e:
        return f"Error retrieving schema: {str(e)}"

# @tool("get_distribution_stats")
# @rate_limit
# def get_distribution_stats(date: str = None) -> str:
#     """
#     Get statistical distribution of firm characteristics at a specific date.
    
#     Args:
#         date: Date in format 'YYYY-MM-DD' (if None, uses latest date)
    
#     Returns:
#         Distribution statistics for all characteristics
#     """
#     try:
#         conn = sqlite3.connect('firm_data.db')
        
#         if date is None:
#             cursor = conn.cursor()
#             cursor.execute("SELECT MAX(datadate) FROM firms")
#             date = cursor.fetchone()[0]
        
#         query = f"""
#         SELECT 
#             COUNT(*) as n_firms,
#             AVG(mve) as avg_mve, MIN(mve) as min_mve, MAX(mve) as max_mve,
#             AVG(bm) as avg_bm, MIN(bm) as min_bm, MAX(bm) as max_bm,
#             AVG(mom12m) as avg_mom12m, MIN(mom12m) as min_mom12m, MAX(mom12m) as max_mom12m
#         FROM firms
#         WHERE datadate = '{date}'
#         """
        
#         df = pd.read_sql_query(query, conn)
#         conn.close()
        
#         if df.empty:
#             return f"No data found for date: {date}"
        
#         result = f"Distribution statistics at {date}:\n\n"
#         result += df.to_string(index=False)
        
#         return result
#     except Exception as e:
#         return f"Error getting distribution: {str(e)}"

@tool("get_extreme_firms")
@rate_limit
def get_extreme_firms(characteristic: str, threshold: float, date: str = None) -> str:
    """
    Find firms with extreme values for a given characteristic.
    
    Args:
        characteristic: Column name (e.g., 'bm', 'mom12m', 'mve')
        threshold: Z-score threshold (e.g., 1.5 for values > 1.5 or < -1.5)
        date: Date in format 'YYYY-MM-DD' (if None, uses latest date)
    
    Returns:
        Firms with extreme values
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        
        if date is None:
            cursor = conn.cursor()
            cursor.execute("SELECT MAX(datadate) FROM firms")
            date = cursor.fetchone()[0]
        
        query = f"""
        SELECT ticker, datadate, mve, bm, mom12m
        FROM firms
        WHERE datadate = '{date}' 
        AND (ABS({characteristic}) > {threshold})
        ORDER BY ABS({characteristic}) DESC
        """
        
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        if df.empty:
            return f"No firms found with |{characteristic}| > {threshold} at {date}"
        
        result = f"Firms with extreme {characteristic} (|z-score| > {threshold}) at {date}:\n\n"
        result += df.to_string(index=False)
        
        return result
    except Exception as e:
        return f"Error finding extreme firms: {str(e)}"


@tool("test_complex_condition")
@rate_limit
def test_complex_condition(sql_condition: str, date: str = None) -> str:
    """
    Test ANY complex condition combining multiple characteristics with AND/OR/NOT logic.
    
    This is the most flexible tool - you can test ANY combination of characteristics.
    
    Examples:
    - "bm < -1.0 AND mom12m > 0.5" (value + momentum)
    - "bm > 1.5 OR mom12m < -1.0" (expensive OR poor momentum)
    - "(bm < -0.8 AND mve > 0.5) OR mom12m > 1.5" (large value stocks OR strong momentum)
    - "bm < -1.0 AND mom12m > 0.5 AND mve > 0" (value + momentum + large cap)
    - "NOT (bm > 1.0 OR mom12m < -0.5)" (exclude expensive or negative momentum)
    - "(bm < -1.2 OR mom12m > 1.0) AND mve > 0" (value OR momentum, in large caps)
    
    Args:
        sql_condition: Any valid SQL WHERE clause using mve, bm, mom12m
        date: Date in format 'YYYY-MM-DD' (if None, uses latest date)
    
    Returns:
        Number of firms meeting the condition and their characteristics
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        
        if date is None:
            cursor = conn.cursor()
            cursor.execute("SELECT MAX(datadate) FROM firms")
            date = cursor.fetchone()[0]
        
        # Count firms meeting condition
        count_query = f"""
        SELECT COUNT(*) as n_firms
        FROM firms
        WHERE datadate = '{date}' 
        AND ({sql_condition})
        AND mve IS NOT NULL AND bm IS NOT NULL AND mom12m IS NOT NULL
        """
        
        count_df = pd.read_sql_query(count_query, conn)
        n_firms = int(count_df['n_firms'].values[0])
        
        # Get sample of firms
        sample_query = f"""
        SELECT ticker, mve, bm, mom12m
        FROM firms
        WHERE datadate = '{date}' 
        AND ({sql_condition})
        AND mve IS NOT NULL AND bm IS NOT NULL AND mom12m IS NOT NULL
        ORDER BY RANDOM()
        LIMIT 20
        """
        
        sample_df = pd.read_sql_query(sample_query, conn)
        
        # Get total firms for percentage
        total_query = f"""
        SELECT COUNT(*) as total
        FROM firms
        WHERE datadate = '{date}'
        AND mve IS NOT NULL AND bm IS NOT NULL AND mom12m IS NOT NULL
        """
        
        total_df = pd.read_sql_query(total_query, conn)
        total_firms = int(total_df['total'].values[0])
        
        conn.close()
        
        percentage = (n_firms / total_firms * 100) if total_firms > 0 else 0
        
        result = f"""Condition Test at {date}:

Condition: {sql_condition}

Results:
  Firms meeting condition: {n_firms:,} ({percentage:.1f}% of {total_firms:,} total firms)

Sample of firms meeting condition:
"""
        if not sample_df.empty:
            result += "\n" + sample_df.to_string(index=False)
        else:
            result += "\n(No firms meet this condition)"
        
        return result
    except Exception as e:
        return f"Error testing condition: {str(e)}\nMake sure your SQL condition is valid."

# ============================================
# Configure LLM
# ============================================
def get_rate_limited_llm():
    """Get LLM provider with rate limiting configured."""
    llm = AG.get_llm_provider()
    try:
        llm.max_retries = 5
        llm.timeout = 60
    except:
        pass
    return llm

# ============================================
# Strategy Development Agent (Single Agent)
# ============================================
strategy_agent = Agent(
    role="Quantitative Strategy Developer",
    goal="Develop systematic BUY/HOLD/SELL rules based on firm characteristics that can be applied to all S&P 500 firms",
    backstory="""
    You are an expert quantitative strategist who creates systematic, rule-based trading strategies.
    
    CRITICAL DATA UNDERSTANDING:
    - 'mve' = log(market value of equity), represents log firm size
    - 'bm' = book-to-market ratio (value factor). Understand that a high book-to-market value means undervalued, and a low book-to-market value means overvalued.
    - 'mom12m' = 12-month momentum
    - ALL features are standardized: mean = 0, standard deviation = 1
    - Values are z-scores showing standard deviations from mean
    
    Your task is to develop EXPLICIT, SYSTEMATIC RULES for generating trading signals. Understand that when doing the following, you must use causal masking from December 2023 to prevent any look-ahead bias.
    
    1. EXPLORE THE DATA (December 2023):
       - Identify what constitutes "extreme" values for mve, bm, and mom12m
       - Look for natural clustering or breakpoints in the data
       - Consider correlations between characteristics
    
    2. DEVELOP CLEAR RULES based on economic intuition:
       - Keep in mind the market conditions at this date - this might influence the rules you choose.
       - The following are example questions you can consider, BUT THEY ARE NOT EXHAUSTIVE: 
           - Value stocks: Should low bm (cheap) be BUY or SELL?
           - Momentum: Should high mom12m (strong performance) be BUY or SELL?
           - Size: Should mve matter for the strategy?
           - Combinations: What about value + momentum + size together?
    
    3. DEFINE SPECIFIC THRESHOLDS:
     Your output must include exact rules. You can use test_complex_condition to test different combinations.
       Your output must include exact rules like:
       - "BUY if: bm < -0.71 AND mom12m > 0.85 AND mve > 0.23" or "BUY if: (bm > 0.57 AND mom12m < 0.82) OR mve > -0.88" or "BUY if: bm > 0.63 OR mve < 0.98"
       - "SELL if: bm > 1.25 OR mom12m < -0.98" or "SELL if: (bm < -0.84 OR mve < 0) AND mom12m < -0.97" or "SELL if: (bm > 0.94 AND mom12m < -1.06) OR mve <-0.68"
       - "HOLD: all other cases"
       - However, the above is only AN EXAMPLE - so do not simply copy the format above. You are free to include/exclude as many conditions in the if statements. You are also free to make the conditions as complicated or as simple as you like.
       - Be PRECISE in your thresholds, do not choose numbers that are nice or round - you are a quantative strategy developer.
    
    4. PROVIDE RATIONALE:
       - Why these thresholds?
       - What's the economic intuition?
       - What patterns did you observe in the data?
    
    CRITICAL REQUIREMENTS:
    - Rules must be DETERMINISTIC (same inputs → same output)
    - Use ONLY z-score comparisons (>, <, AND, OR), but these may be impacted by market conditions.
    - Define thresholds for BUY, SELL, and HOLD
    - Rules should be implementable as: if (condition) then signal
    - Keep in mind that we will use the buy signals to construct a portfolio, so it is better to give too many signals, rather than too few signals.
    
    OUTPUT FORMAT:
    ===========================================
    SYSTEMATIC TRADING RULES
    ===========================================
    
    Data Exploration Summary:
    - [Key statistics and patterns observed]
    
    BUY RULE:
    if [ANY complex z-score condition using AND/OR/NOT]:
        signal = BUY
    
    Examples of valid BUY rules:
    - "bm < -1.15 AND mom12m > 0.73 AND mve < 1.11" (simple AND)
    - "bm < -1.56 OR mom12m > 1.28 OR mve > 1.52" (simple OR)
    - "(bm < -1.02 AND mom12m > 0.53) OR mve > 1.59" (combination)
    - "bm < -0.83 AND (mom12m > 0.77 OR mve > 1.08)" (nested conditions)
    
    SELL RULE:
    if [ANY complex z-score condition]:
        signal = SELL
    
    HOLD RULE:
    else:
        signal = HOLD
    
    Rationale:
    - [Economic reasoning for BUY rule]
    - [Economic reasoning for SELL rule]
    - [Expected signal distribution]
    
    ```
    ===========================================
    
    Be precise, systematic, and data-driven. Your rules will be applied to ~500 firms.
    """,
    llm=get_rate_limited_llm(),
    memory=True,
    verbose=True,
    max_iter=15,
    max_rpm=10
)

strategy_task = Task(
    description="""
    Develop systematic BUY/HOLD/SELL rules for S&P 500 firms at December 2023.
    
    Available characteristics (all are z-scores):
    - mve: log firm size
    - bm: log book-to-market (value)
    - mom12m: 12-month momentum
    
    Your process:
    1. Get database schema and understand available data for December 2023
    2. Explore extreme values for each characteristic
    3. Look for patterns and correlations
    4. Use test_complex_condition to test different rule combinations
    5. Develop systematic rules with specific z-score thresholds

    You have COMPLETE FLEXIBILITY in creating rules. Test different combinations using AND, OR, NOT.
    
    CRITICAL: Your output must be EXPLICIT RULES with exact thresholds that can be 
    implemented in Python/pandas. You must give PRECISE THRESHOLDS - do not give thresholds that are only nice or round numbers. Further, use causal masking from December 2023 to prevent any look-ahead bias.
    
    Focus on:
    - Economic intuition (value, momentum, size effects)
    - Clear, implementable thresholds
    - Balance between signal strength and diversification
    - Rules that make sense for ~500 firms
    
    Output systematic rules that I can directly implement in code.
    """,
    expected_output="""
    Complete strategy document with:
    1. Data exploration summary
    2. Explicit BUY rule with z-score thresholds
    3. Explicit SELL rule with z-score thresholds  
    4. HOLD rule (default case)
    5. Economic rationale for each rule
    
    Rules must be deterministic and implementable.
    """,
    agent=strategy_agent,
    tools=[
        get_database_schema,
        query_firm_database,
        get_extreme_firms,
        test_complex_condition
    ]
)

crew = Crew(
    agents=[strategy_agent],
    tasks=[strategy_task],
    process=Process.sequential,
    memory=False,
    verbose=True,
    max_rpm=10
)

# ============================================
# Main Execution
# ============================================
if __name__ == "__main__":

    # selected_cols = variables  # Add your columns
    # setup_firm_database('green cleaned.csv', selected_columns=selected_cols)
    
    print("=" * 70)
    print("SYSTEMATIC STRATEGY DEVELOPER")
    print("=" * 70)
    print("\nGenerates systematic BUY/HOLD/SELL rules for S&P 500 firms")
    print("Based on z-score analysis of firm characteristics")
    print("\nThe agent will:")
    print("  1. Explore your firm characteristics data")
    print("  2. Develop systematic rules with specific thresholds")
    print("  3. Output implementation-ready strategy")
    print("\nYou can then apply the rules to all firms manually.")
    print("=" * 70)
    
    input("\nPress Enter to start strategy development...")
    
    try:
        print("\n🚀 Starting strategy development...\n")
        result = crew.kickoff()
        
        print("\n" + "=" * 70)
        print("SYSTEMATIC STRATEGY OUTPUT")
        print("=" * 70)
        print(result)
        print("=" * 70)
        
        # Save to file
        with open('systematic_strategy_2024.txt', 'w') as f:
            f.write(str(result))
        
        print("\n✓ Strategy saved to 'systematic_strategy_2024.txt'")
        print("\nNext steps:")
        print("  1. Review the strategy rules")
        print("  2. Implement in Python/pandas using the thresholds")
        print("  3. Apply to all firms in your dataset")
        print("  4. Generate signals for backtesting")
        
    except KeyboardInterrupt:
        print("\n\n⏸ Interrupted.")
    except Exception as e:
        print(f"\n❌ Error: {e}")
        if "429" in str(e) or "RateLimitError" in str(e):
            print("\n🚨 Rate limit hit. Wait 60 seconds and try again.")

SYSTEMATIC STRATEGY DEVELOPER

Generates systematic BUY/HOLD/SELL rules for S&P 500 firms
Based on z-score analysis of firm characteristics

The agent will:
  1. Explore your firm characteristics data
  2. Develop systematic rules with specific thresholds
  3. Output implementation-ready strategy

You can then apply the rules to all firms manually.



Press Enter to start strategy development... 



🚀 Starting strategy development...



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 82941523-ddce-4657-94c4-c5cab62ed232                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Quantitative Strategy Developer                                                                         │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Develop systematic BUY/HOLD/SELL rules for S&P 500 firms at December 2023.                                 │
│                                                                                                                 │
│      Available characteristics (all are z-scores):                                                              │
│      - mve: log firm size                                                                                       │
│      - bm: log book-to-market (value)                                                                           │
│      - mom12m: 12-month momentum                                                                                │
│                                                                                                                 │
│      Your process:                                                                                              │
│      1. Get database schema and understand available data for December 2023                                     │
│      2. Explore extreme values for each characteristic                                                          │
│      3. Look for patterns and correlations                                                                      │
│      4. Use test_complex_condition to test different rule combinations                                          │
│      5. Develop systematic rules with specific z-score thresholds                                               │
│                                                                                                                 │
│      You have COMPLETE FLEXIBILITY in creating rules. Test different combinations using AND, OR, NOT.           │
│                                                                                                                 │
│      CRITICAL: Your output must be EXPLICIT RULES with exact thresholds that can be                             │
│      implemented in Python/pandas. You must give PRECISE THRESHOLDS - do not give thresholds that are only      │
│  nice or round numbers. Further, use causal masking from December 2023 to prevent any look-ahead bias.          │
│                                                                                                                 │
│      Focus on:                                                                                                  │
│      - Economic intuition (value, momentum, size effects)                                                       │
│      - Clear, implementable thresholds                                                                          │
│      - Balance between signal strength and diversification                                                      │
│      - Rules that make sense for ~500 firms                                                                     │
│                                                                                                                 │
│      Output systematic rules that I can directly implement in code.                                             │
│                                                                                                                 │
│                                                                                                                 │
╰────────────────────────────────────────────────────────

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Quantitative Strategy Developer                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  ===========================================                                                                    │
│  SYSTEMATIC TRADING RULES                                                                                       │
│  ===========================================                                                                    │
│                                                                                                                 │
│  Data Exploration Summary:                                                                                      │
│  - The data includes standardized book-to-market (bm), momentum (mom12m), and market value of equity (mve) for  │
│  S&P 500 firms as of December 2023.                                                                             │
│  - All features are standardized to have a mean of 0 and a standard deviation of 1.                             │
│  - Extreme values were explored to understand the distribution of characteristics.                              │
│                                                                                                                 │
│  BUY RULE:                                                                                                      │
│  if bm > 0.95 AND mve > 0.3 AND mom12m > -0.5:                                                                  │
│      signal = BUY                                                                                               │
│                                                                                                                 │
│  SELL RULE:                                                                                                     │
│  if (bm < -0.75 OR mom12m < -0.55 OR mve < -0.75) AND NOT (mom12m > 1.5):                                       │
│      signal = SELL                                                                                              │
│                                                                                                                 │
│  HOLD RULE:                                                                                                     │
│  else:                                                                                                          │
│      signal = HOLD                                                                                              │
│                                                                                                                 │
│  Rationale:                                                                                                     │
│  - BUY Rule: This rule targets undervalued (high bm), reasonably sized (mve > 0.3) companies with positive      │
│  momentum (mom12m > -0.5). The economic intuition is to buy companies that are currently cheap but have shown   │
│  some signs of recovery or positive market sentiment.                                                           │
│  - SELL Rule: This rule aims to sell companies that are overvalued (low bm), have negative momentum (mom12m <   │
│  -0.55), or are small in size (mve < -0.75). The `NOT (mom12m > 1.5)` condition prevents selling companies      │
│  with extremely high momentum, even if they meet the ot

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5b88e8f9-61a2-4764-8892-090c14da0fec                                                                     │
│  Agent: Quantitative Strategy Developer                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 82941523-ddce-4657-94c4-c5cab62ed232                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```                                                                                              │
│  ===========================================                                                                    │
│  SYSTEMATIC TRADING RULES                                                                                       │
│  ===========================================                                                                    │
│                                                                                                                 │
│  Data Exploration Summary:                                                                                      │
│  - The data includes standardized book-to-market (bm), momentum (mom12m), and market value of equity (mve) for  │
│  S&P 500 firms as of December 2023.                                                                             │
│  - All features are standardized to have a mean of 0 and a standard deviation of 1.                             │
│  - Extreme values were explored to understand the distribution of characteristics.                              │
│                                                                                                                 │
│  BUY RULE:                                                                                                      │
│  if bm > 0.95 AND mve > 0.3 AND mom12m > -0.5:                                                                  │
│      signal = BUY                                                                                               │
│                                                                                                                 │
│  SELL RULE:                                                                                                     │
│  if (bm < -0.75 OR mom12m < -0.55 OR mve < -0.75) AND NOT (mom12m > 1.5):                                       │
│      signal = SELL                                                                                              │
│                                                                                                                 │
│  HOLD RULE:                                                                                                     │
│  else:                                                                                                          │
│      signal = HOLD                                                                                              │
│                                                                                                                 │
│  Rationale:                                                                                                     │
│  - BUY Rule: This rule targets undervalued (high bm), reasonably sized (mve > 0.3) companies with positive      │
│  momentum (mom12m > -0.5). The economic intuition is to buy companies that are currently cheap but have shown   │
│  some signs of recovery or positive market sentiment.                                                           │
│  - SELL Rule: This rule aims to sell companies that are overvalued (low bm), have negative momentum (mom12m <   │
│  -0.55), or are small in size (mve < -0.75). The `NOT 


SYSTEMATIC STRATEGY OUTPUT
```
SYSTEMATIC TRADING RULES

Data Exploration Summary:
- The data includes standardized book-to-market (bm), momentum (mom12m), and market value of equity (mve) for S&P 500 firms as of December 2023.
- All features are standardized to have a mean of 0 and a standard deviation of 1.
- Extreme values were explored to understand the distribution of characteristics.

BUY RULE:
if bm > 0.95 AND mve > 0.3 AND mom12m > -0.5:
    signal = BUY

SELL RULE:
if (bm < -0.75 OR mom12m < -0.55 OR mve < -0.75) AND NOT (mom12m > 1.5):
    signal = SELL

HOLD RULE:
else:
    signal = HOLD

Rationale:
- BUY Rule: This rule targets undervalued (high bm), reasonably sized (mve > 0.3) companies with positive momentum (mom12m > -0.5). The economic intuition is to buy companies that are currently cheap but have shown some signs of recovery or positive market sentiment.
- SELL Rule: This rule aims to sell companies that are overvalued (low bm), have negative momentum (mom12m < -0.5